## Story Cloze: Bert for Sequence Classification

#### install tools

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.6 MB/s eta 0:00:00


In [2]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

import datasets
from datasets import load_dataset, load_metric
from datasets import Dataset


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# load training, validation, and test datasets
train_df = pd.read_json('/content/drive/MyDrive/Project/ROCStories_winter2017_mixed.json')
val_df = pd.read_csv('/content/drive/MyDrive/Project/cloze_test_val__winter2018-cloze_test_ALL_val - 1 - 1.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Project/cloze_test_test__winter2018-cloze_test_ALL_test - 1.csv')

### Modify Training Test and Validation Datasets

In [6]:
def modify_training(data):
  new = pd.DataFrame()
  new['stories'] = data.story
  new['ending0'] = data.ending0
  new['ending1'] = data.ending1
  new['labels'] = data.label
  ntest = new.melt(id_vars=['stories','labels'], value_vars=['ending0', 'ending1'], var_name='ending_number', value_name='endings')
  check_last_index_match = lambda row: 1 if row['ending_number'][-1] == str(row['labels']) else 0
  ntest['label']=ntest.apply(check_last_index_match, axis=1)
  ntest.drop(['labels','ending_number'],inplace=True,axis=1)
  return ntest.sample(frac=1,replace=False).reset_index(drop=True)

def modify_validation(val_df):
  validation = pd.DataFrame()
  validation['stories'] = val_df.iloc[:,1:5].apply(lambda row: ' '.join(row), axis=1)
  validation['ending1'] = val_df.iloc[:,-3]
  validation['ending2'] = val_df.iloc[:,-2]
  validation['labels'] = val_df.iloc[:,-1]
  vtest = validation.melt(id_vars=['stories','labels'], value_vars=['ending1', 'ending2'], var_name='ending_number', value_name='endings')
  check_last_index_match = lambda row: 1 if row['ending_number'][-1] == str(row['labels']) else 0
  vtest['label']=vtest.apply(check_last_index_match, axis=1)
  vtest.drop(['labels','ending_number'],inplace=True,axis=1)
  return vtest.sample(frac=1,replace=False).reset_index(drop=True)



In [7]:
### turn all train, validation, and test datasets into a dataset dictionary ###
test_df['label'] = [-1] * len(test_df)

train_mod = modify_training(train_df)
val_mod = modify_validation(val_df)
test_mod = modify_validation(test_df)
test_mod['label'] = [-1] * len(test_mod)


train_dataset = Dataset.from_dict(train_mod)
val_dataset = Dataset.from_dict(val_mod)
test_dataset = Dataset.from_dict(test_mod)
data_dict = datasets.DatasetDict({"train":train_dataset,
                                        "validation" : val_dataset,
                                        "test":test_dataset})

## Look at some examples

In [8]:
def show_one(example):
    print(f"Story: {example['stories']}")
    print(f"\nEnding: {example['endings']}")
    print(f"\nLabel: {example['label']}")

In [9]:
show_one(data_dict['train'][9])

Story: Kia went to her son's soccer game. She sat in the bleachers to cheer him on. But then a soccer ball flew into the stands! It hit Kia in the cheek, causing a big bruise.

Ending: Kia decided to sit farther back next time!

Label: 1


In [10]:
show_one(data_dict['validation'][100])

Story: Damon lost his wallet with his ID in it. He searched all over the house. He couldn't find it anywhere. He eventually just went and got a replacement ID.

Ending: Damon went straight to his room and went to bed.

Label: 0


## Preprocess

In [11]:
### load tokenizer ###
from transformers import AutoTokenizer
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

#### Tokenize each sentence

In [12]:
def preprocess_function(examples):
    return tokenizer(examples['stories'], examples['endings'], truncation=True, padding='longest',return_tensors='tf')

In [13]:
pre_tokenizer_columns = set(data_dict["train"].features)
encoded_dataset = data_dict.map(preprocess_function, batched=True)
tokenizer_columns = list(set(encoded_dataset["train"].features) - pre_tokenizer_columns)
print("Columns added by tokenizer:", tokenizer_columns)

Map:   0%|          | 0/105330 [00:00<?, ? examples/s]

Map:   0%|          | 0/3142 [00:00<?, ? examples/s]

Map:   0%|          | 0/3142 [00:00<?, ? examples/s]

Columns added by tokenizer: ['input_ids', 'attention_mask', 'token_type_ids']


#### Look at some tokenization and preprocessing outputs

In [14]:
examples = data_dict["train"][:3]
features = preprocess_function(examples)
print(
    len(features["input_ids"]),
    len(features["input_ids"][0]),
    [len(x) for x in features["input_ids"]],
)

3 61 [61, 61, 61]


In [15]:
[tokenizer.decode(features["input_ids"][x]) for x in range(len(features))]

["[CLS] maddy was very clumsy. she tripped over everything and made people laugh. as she was very self - conscious, she tried to be careful. she took lessons to have more poise and walk with a straight back. [SEP] now, people look at her with awe and don't laugh anymore. [SEP]",
 '[CLS] tom noticed a spider web near his bed. he looked around for a spider. tom never found one. he destroyed the web. [SEP] donald went on to become the president of the united states of america. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 "[CLS] jimmy was worried. he forgot where he'd put his pills. he searched all over for them, with no luck. that's when he found his prescription. [SEP] jimmy read it and realized he wasn't due for another set for a day. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"]

## Load Model

In [16]:
from transformers import TFBertForSequenceClassification

id2label = {0: "Invalid", 1: "Valid"}
label2id = {val: key for key, val in id2label.items()}

model = TFBertForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
tf_train_dataset = model.prepare_tf_dataset(
    encoded_dataset["train"],
    shuffle=True,
    batch_size=16,
    tokenizer=tokenizer
)

tf_validation_dataset = model.prepare_tf_dataset(
    encoded_dataset["validation"],
    shuffle=False,
    batch_size=16,
    tokenizer=tokenizer,
)

#### Evaluation Metrics

In [18]:
import evaluate
accuracy = evaluate.load("accuracy")
#recall = evaluate.load("recall")

In [19]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [22]:
from transformers import create_optimizer

num_epochs = 3
batch_size = 16
batches_per_epoch = len(encoded_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=3e-05, num_warmup_steps=0, num_train_steps=total_train_steps
)
model.compile(optimizer=optimizer)

In [23]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=tf_validation_dataset
)

In [24]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs,
    callbacks=[metric_callback],
)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
6583/6583 [==============================] - 2206s 322ms/step - loss: 0.1039 - val_loss: 2.8093 - accuracy: 0.5328
Epoch 2/3
6583/6583 [==============================] - 2085s 317ms/step - loss: 0.0323 - val_loss: 3.6671 - accuracy: 0.5388
Epoch 3/3
6583/6583 [==============================] - 2093s 318ms/step - loss: 0.0072 - val_loss: 4.1436 - accuracy: 0.5407


In [50]:
index = np.random.randint(0, len(test_df)-2)
sentences = [
    test_df.iloc[index:index+1,1:6].apply(lambda row: ' '.join(row), axis=1)[index],
    test_df.iloc[index:index+1,[1,2,3,4,6]].apply(lambda row: ' '.join(row), axis=1)[index]
             ]
print(sentences[0])
print(sentences[1])

Adam was in history class. He was bored. Adam carved his name into his desk. Adam was caught. He left class and went home.
Adam was in history class. He was bored. Adam carved his name into his desk. Adam was caught. He had to stay after school and sand the desk.


In [52]:
tokenized = tokenizer(sentences, return_tensors="tf", padding="longest")

logits = model(tokenized).logits
print(logits)
classifications = np.argmax(logits, axis=1)
print(classifications)

classifications = [model.config.id2label[output] for output in classifications]
print(classifications)

tf.Tensor(
[[ 0.04125177 -0.2760779 ]
 [-0.5525273   0.5601171 ]], shape=(2, 2), dtype=float32)
[0 1]
['Invalid', 'Valid']


In [57]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [60]:
model.save('/content/drive/MyDrive/Project/model.keras')

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [68]:
my_model = tf.keras.models.load_model('/content/drive/MyDrive/Project/model.keras')

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [73]:
my_model.compile(optimizer=optimizer)
my_model.build()
my_model.summary()

Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_113 (Dropout)       multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
